In [73]:
from sklearn.cluster import KMeans
import numpy as np
import json
import os
import pickle

def load_json(path):
    f=open(path,"r")
    return json.load(f)

def write_json(content,out):
    f=open(out,"w")
    f.write(json.dumps(content, sort_keys=False, indent=2))
    f.close()
    
def load_pickle(file):
    with open(file, "rb") as f:
        return pickle.load(f)

In [89]:
#prefix="/media/jialu/Backup Plus/KIC-PH2/KIC_files_15832/short_wav_all/features/noi_label"
prefix="/media/jialu/Backup Plus/IDP_speech/features/noi_label"
json_data=load_json(os.path.join(prefix,"total_no_noi.json"))
keys=sorted(json_data.keys())

curr_trans_data={}

def check_segment(line_prev,line_curr):
    return line_prev[1]==line_curr[1] and line_prev[-3]==line_curr[-3] and line_prev[-4]==line_curr[-4]

for i in range(len(json_data)-1):
    prev,curr=i,i+1
    prev_line=keys[prev].split("_")
    curr_line=keys[curr].split("_")
    if check_segment(prev_line, curr_line):
        prev_start,curr_start=float(prev_line[-2]),float(curr_line[-2])
        if abs(curr_start-prev_start)>0.2: continue
        new_trans_key="_".join(prev_line[:-1])+"_"+curr_line[-2]
        curr_trans_data[new_trans_key]={
                "wav_prev":json_data[keys[prev]]["wav"],
                "wav_curr":json_data[keys[curr]]["wav"],
                "w2v2_prev":json_data[keys[prev]]["w2v2_conv"],
                "w2v2_curr":json_data[keys[curr]]["w2v2_conv"],
                "sp_prev":json_data[keys[prev]]["sp"],
                "sp_curr":json_data[keys[curr]]["sp"]                
            }
write_json(curr_trans_data, os.path.join(prefix,"testing_conv_kmeans_sample.json"))
    

In [167]:
#prefix="/media/jialu/Backup Plus/KIC-PH2/KIC_files_15832/short_wav_all/features/noi_label"
prefix="/media/jialu/Backup Plus/IDP_speech/features/noi_label"

json_data=load_json(os.path.join(prefix,"total_no_noi.json"))
keys=sorted(json_data.keys())
skip=100
curr_trans_data={}

def check_segment(lines):
    for i in range(len(lines)-1):
        if lines[i][0]!=lines[i+1][0] or lines[i][-3]!=lines[i+1][-3] or lines[i][-4]!=lines[i+1][-4]:
            return False
    return True
for i in range(len(json_data)-skip):
#for i in range(50000):
    if check_segment([keys[k].split("_") for k in range(i,i+skip+1)]): 
        first_line=keys[i].split("_")
        last_line=keys[i+skip].split("_")
        prev_start,curr_start=float(first_line[-2]),float(last_line[-2])
        if abs(curr_start-prev_start)>0.2*skip: continue
        new_trans_key="_".join(first_line[:-1])+"_"+last_line[-2]
        curr_trans_data[new_trans_key]={}
        for j in range(i,i+skip):
            curr_trans_data[new_trans_key]["w2v2_conv_{}".format(j-i)]=json_data[keys[j]]["w2v2_conv"]
            curr_trans_data[new_trans_key]["sp_{}".format(j-i)]=json_data[keys[j]]["sp"]

write_json(curr_trans_data, os.path.join(prefix,"testing_conv_kmeans_sample_{}utt.json".format(skip)))
    

In [100]:
prefix="/media/jialu/Backup Plus/KIC-PH2/KIC_files_15832/short_wav_all/features/noi_label"
json_data_sample=load_json(os.path.join(prefix,"testing_conv_kmeans_sample.json"))
    
output_label_map={}
for sp1 in ["SIL","CHN","FAN","MAN","CXN"]:
    for sp2 in ["SIL","CHN","FAN","MAN","CXN"]:
        label=None
        if sp1=="SIL" and sp2=="SIL": label="SIL_SIL"
        elif sp1=="SIL" and sp2=="CHN": label="SIL_CHN"
        elif sp1=="CHN" and sp2=="SIL": label="CHN_SIL"
        elif sp1=="SIL" and sp2!="CHN": label="SIL_FMC"
        elif sp1!="CHN" and sp2=="SIL": label="FMC_SIL"
        elif sp1=="CHN" and sp2!="CHN": label="CHN_FMC"
        elif sp1!="CHN" and sp2=="CHN": label="FMC_CHN"
        elif sp1=="CHN" and sp2=="CHN": label="CHN_CHN"    
        elif sp1!="CHN" and sp2!="CHN": label="FMC_FMC"         
        if not label:
            print(label,sp1,sp2)
        output_label_map[sp1+"_"+sp2]=label
        
def compute_feature(w2v1,w2v2,method="concat"):
    if method=="concat": return np.r_[w2v1,w2v2]
    if method=="mean": return np.stack((w2v1,w2v2)).mean(0)
    return None
features=[]
labels_raw=[]
labels=[]
for key,entry in json_data_sample.items():
    #if entry["sp_prev"]=="SIL" and entry["sp_curr"]=="SIL": continue
    w2v2_prev=load_pickle(entry["w2v2_prev"])
    w2v2_curr=load_pickle(entry["w2v2_curr"])
    feature=compute_feature(w2v2_prev,w2v2_curr,method="concat")
    if np.any(np.isnan(feature)): continue
    features.append(feature)
    labels_raw.append(entry["sp_prev"]+"_"+entry["sp_curr"])
    labels.append(output_label_map[entry["sp_prev"]+"_"+entry["sp_curr"]])


In [168]:
skip=100
#prefix="/media/jialu/Backup Plus/KIC-PH2/KIC_files_15832/short_wav_all/features/noi_label"
prefix="/media/jialu/Backup Plus/IDP_speech/features/noi_label"

json_data_sample=load_json(os.path.join(prefix,"testing_conv_kmeans_sample_{}utt.json".format(skip)))
           
def compute_feature(w2vs,method="concat"):
    if method=="concat": return np.stack((w2vs)).reshape(1,-1)
    if method=="mean": return np.stack((w2vs)).mean(0)
    return None

features=[]
labels=[]
labels_no_dup=[]
def get_output_label(entry,no_dup=False):
    label=entry["sp_0"]
    if entry["sp_0"]=="FAN" or entry["sp_0"]=="MAN" or entry["sp_0"]=="CXN":
        label="FMC"
    prev_label=label
    for i in range(1,skip):
        curr_label=entry["sp_{}".format(i)]
        if entry["sp_{}".format(i)]=="FAN" or entry["sp_{}".format(i)]=="MAN" or entry["sp_{}".format(i)]=="CXN":
            curr_label="FMC"
        if no_dup:
            if curr_label!=prev_label:
                label+="_"+curr_label
                prev_label=curr_label
        else:
                label+="_"+curr_label            
    return label

for key,entry in json_data_sample.items():
    curr_features=[]
    for i in range(skip):
        curr_features.append(load_pickle(entry["w2v2_conv_{}".format(i)]))
    feature=compute_feature(curr_features,method="mean")
    if np.any(np.isnan(feature)): continue
    features.append(feature)
    labels.append(get_output_label(entry))
    labels_no_dup.append(get_output_label(entry,no_dup=True))

In [169]:
Counter(labels_no_dup).most_common()[:20]
#Counter(labels).most_common()[:20]

[('SIL', 2115),
 ('SIL_CHN_SIL_CHN_SIL', 733),
 ('SIL_CHN_SIL', 657),
 ('SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL', 629),
 ('SIL_CHN_SIL_CHN_SIL_CHN_SIL', 415),
 ('SIL_FMC_SIL_FMC_SIL_FMC_SIL', 412),
 ('SIL_FMC_SIL', 407),
 ('SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL', 386),
 ('SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL', 386),
 ('SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL', 374),
 ('SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL', 336),
 ('SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL', 306),
 ('SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL', 297),
 ('SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL', 292),
 ('SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL', 213),
 ('CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL', 207),
 ('SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN', 201),
 ('CHN_SIL_CHN_SIL_CHN_SIL_CHN', 196),
 ('CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN', 187),
 ('SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN', 187)]

In [170]:
kmeans=KMeans(n_clusters=4,random_state=0).fit(features)


In [171]:
labels_count={}
top_cate=5
for i,l in enumerate(kmeans.labels_):
    if l not in labels_count:
        labels_count[l]=[]
    labels_count[l].append(labels_no_dup[i])
for k,v in labels_count.items():
    count=Counter(v)
    total_number=sum(count.values())
    print("---------------------")
    percentage=[round(v/total_number*100,2) for (k,v) in count.most_common()[:top_cate]]
    for j,l in enumerate(count.most_common()[:top_cate]):
        print(l,percentage[j])

---------------------
('SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL', 284) 5.49
('SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL', 246) 4.75
('SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL', 209) 4.04
('SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL', 203) 3.92
('SIL_FMC_SIL_FMC_SIL_FMC_SIL_FMC_SIL', 178) 3.44
---------------------
('SIL', 1932) 29.15
('SIL_CHN_SIL', 657) 9.91
('SIL_CHN_SIL_CHN_SIL', 621) 9.37
('SIL_FMC_SIL', 382) 5.76
('SIL_FMC_SIL_FMC_SIL_FMC_SIL', 297) 4.48
---------------------
('SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL', 380) 11.57
('SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL', 232) 7.06
('SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL', 217) 6.61
('SIL_CHN_SIL_CHN_SIL_CHN_SIL', 209) 6.36
('SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL', 199) 6.06
---------------------
('CHN_SIL_CHN_SIL_CHN_SIL_CHN', 163) 7.13
('CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN', 148) 6.47
('SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_SIL_CHN_S